In [15]:
import linecache
from symspellpy import SymSpell, Verbosity
from rich import print
import inspect
import os
from collections import Counter
import numpy as np

In [16]:
CONFIG_DIR = '/home/silvio/miniconda3/envs/classy3/prg/config/'
STOPWORD_ES = 'stopwords_es.txt'
STOPWORD_RED = 'stopwords_reddit.txt'
DICTIONARY = 'new_dic.txt'

dictionary = os.path.join(CONFIG_DIR, DICTIONARY)
stopwords_files = [os.path.join(CONFIG_DIR, STOPWORD_ES),
                  os.path.join(CONFIG_DIR,STOPWORD_RED)]
# stop_words_es = os.path.join(CONFIG_DIR, STOPWORD_ES)
# stop_words_red = os.path.join(CONFIG_DIR,STOPWORD_RED)


In [17]:
SS_DISTANCE = 1
SS_VERBOSITY = Verbosity.CLOSEST
RANGES = 10
ss = SymSpell(max_dictionary_edit_distance=SS_DISTANCE, prefix_length=7)
ss.load_dictionary(dictionary, term_index=0, count_index=1)


True

In [18]:
def dic_terms() -> int:
    with open(dictionary, "rbU") as f:
        num_lines = sum(1 for _ in f)
    return num_lines

In [19]:
num_dictionary_terms = dic_terms()
print(f"{num_dictionary_terms=}")

num_dictionary_terms=134839

In [20]:
dictionary_block_size = num_dictionary_terms // RANGES
print(f"{dictionary_block_size=}")

dictionary_block_size=13483

In [21]:
def read_lines_from_file(file_path, line_numbers):
    lines = []
    for line_number in line_numbers:
        line = linecache.getline(file_path, line_number)
        if line:
            lines.append(line.strip())  # Remove leading/trailing whitespace
    return lines

In [22]:
def get_frequency(line_num, file=dictionary):
    line = linecache.getline(file, line_num)  # Fix the order of arguments
    if line:
        words = line.split()
        if len(words) >= 2:
            second_word = words[1]
            try:
                second_word_as_int = int(second_word)
            except ValueError:
                print("Second word is not an integer.")
        else:
            print("Line doesn't contain at least two words.")
    return second_word_as_int

In [23]:
def read_all_stopwords(files=stopwords_files):
    # Initialize an empty Counter
    stopwords = Counter()

    # Iterate through the list of file paths
    for file_path in files:
        try:
            # Open and read the file
            with open(file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
                # Update the Counter with words (stripped of whitespace and newline characters)
                for line in lines:
                    word = line.strip()
                    if word:
                        stopwords.update([word.lower()])
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"An error occurred while reading {file_path}: {str(e)}")

    return stopwords


In [24]:
stopwords = read_all_stopwords()

In [25]:
print(f"{len(stopwords)=}")

len(stopwords)=710

In [26]:
line_numbers = []
thresholds = []
for i in range(1, RANGES):
    line_numbers.append(dictionary_block_size * i)
    thresholds.append(get_frequency(dictionary_block_size * i, file=dictionary))

line_numbers.append(num_dictionary_terms)
thresholds.append(get_frequency(dic_terms(), file=dictionary))

print(line_numbers)
print(thresholds)
  

[13483, 26966, 40449, 53932, 67415, 80898, 94381, 107864, 121347, 134839]

[226935, 81631, 42448, 26122, 17958, 12335, 8599, 6530, 4897, 1632]

In [27]:
thresholds_arr = np.array(thresholds) 
def classify_and_average(numbers):
    """
    Never mind if the spelling become longer!!!
    """
    # numbers is a list 
    # Create a numpy array from the input list of numbers
    numbers_arr = np.array(numbers)
    
    # Use numpy's digitize function to classify the numbers
    classes = np.digitize(numbers_arr, thresholds_arr)
    
    # Calculate the average of the classified results
    average_class = sum(classes) / len(classes)
    
    return average_class

In [35]:
# Define a function to provide suggestions along with frequencies
def suggest_with_frequencies(input_word):
    suggestions = ss.lookup(input_word, Verbosity.CLOSEST, max_edit_distance=1)
    print(suggestions)
    try:
        return suggestions[0].term, suggestions[0].count
    except IndexError:
        # Handle the IndexError gracefully (e.g., return a default value)
        return "No suggestions found", 0


In [31]:
# Example usage:
input_word = "xistren"

In [36]:
suggested_words = suggest_with_frequencies(input_word)
print(suggested_words)

[]

('No suggestions found', 0)

In [ ]:
sentence = "Esta hes una frase de exemplo"